# Задание 1

## Классификация текстов: спам-фильтр для SMS

Считаем датасет в Python.

In [4]:
import numpy as np
data = np.genfromtxt('SMSSpamCollection.txt', dtype=None, delimiter='\t')

Подготовим два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. Метка 1 соответствует спаму, метка 0 - "не спаму".

In [5]:
textdata = [[0 if elem[0].decode('utf8') == 'ham' else 1, elem[1].decode('utf8')] for elem in data]
splitdata = [list(l) for l in zip(*textdata)]
target, texts = splitdata

Получим матрицу признаков X при помощи CountVectorizer из библиотеки sklearn.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
sfecv = CountVectorizer()
X = sfecv.fit_transform(texts)

Используем логистическую регрессию для классификации данного набора текстов.

In [7]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=241)

Оценим качество классификации при помощи кросс-валидации с 10 фолдами.

In [8]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lrc, X, target, scoring='f1', cv=10)
print('Mean score:', scores.mean())

Mean score: 0.933419572279


Обучим классификатор на всей выборке и спрогнозируем с его помощью класс для следующих сообщений:

In [9]:
messages = [
    "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
    "FreeMsg: Txt: claim your reward of 3 hours talk time",
    "Have you visited the last lecture on physics?", 
    "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", 
    "Only 99$"
]
lrc.fit(X, target)
res = lrc.predict(sfecv.transform(messages))
print(res)

[1 1 0 0 0]


Исследуем качество классификации при помощи линейного классификатора при использовании биграмм, триграмм и n-грамм при n=[1,3].

In [10]:
ngrammcombs = [(1,1),(2,2), (3,3), (1,3)]
LR_ngscores = []
for ng_range in ngrammcombs:
    vect = CountVectorizer(ngram_range=ng_range)
    X = vect.fit_transform(texts)
    LR_ngscores.append(cross_val_score(lrc, X, target, scoring='f1', cv=10).mean())

# Ответ
print([float("{0:0.2f}".format(i)) for i in LR_ngscores])

[0.93, 0.82, 0.72, 0.92]


Используем наивный Байес вместо логистической регрессии.

In [11]:
from sklearn.naive_bayes import MultinomialNB
nbc = MultinomialNB()

NB_ngscores = []
for ng_range in ngrammcombs:
    vect = CountVectorizer(ngram_range=ng_range)
    X = vect.fit_transform(texts)
    NB_ngscores.append(cross_val_score(nbc, X, target, scoring='f1', cv=10).mean())

# Ответ
print([float("{0:0.2f}".format(i)) for i in NB_ngscores])

[0.93, 0.64, 0.38, 0.89]


Используем в качестве признаков в логистической регрессии Tf*idf из Tfidfvectorizer на униграммах.

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfv = TfidfVectorizer(ngram_range=(1,1))
X = tfidfv.fit_transform(texts)
print('Mean score: {0:0.2f}.'.format(cross_val_score(lrc, X, target, scoring='f1', cv=10).mean()))

Mean score: 0.85.


Качество на кросс-валидации понизилось (0.85 < 0.93) по сравнению с CountVectorizer() на униграммах.

### Попробуем повысить качество на кросс-валидации.

1) Попробуем осуществить поиск оптимальных параметров для коэффициента и типа регуляризации. Это может несколько повысить качество классификации.

Используем результат CountVectorizer() в качестве признаков для логистической регрессии.

In [13]:
from sklearn.model_selection import GridSearchCV

X = sfecv.fit_transform(texts)
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [1e-6, 1e-4, 1e-2, 1, 1e2, 1e4, 1e6]
    }
gscv = GridSearchCV(lrc, param_grid, cv=10)
gscv.fit(X,target)
print(gscv.best_params_)
print('Mean score:', cross_val_score(gscv.best_estimator_, X, target, scoring='f1', cv=10).mean())

{'C': 100.0, 'penalty': 'l2'}
Mean score: 0.944157839942


In [14]:
res = gscv.best_estimator_.predict(sfecv.transform(messages))
print(res)

[1 1 0 0 0]


По результатам GridSearchCV можно сделать вывод, что первоначальная регуляризация в линейной модели была черезмерно строга, поскольку увеличение коэффициента C (уменьшение коэффициента регуляризации) привело к повышению качества на кросс-валидации.

2) Предобработка текста

In [16]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

Методы:
- приведение к нижнему регистру
- удалние стоп-слов
- лемматизация
- стэмминг



In [19]:
processed_texts = [(' '.join([ps.stem(lemmatizer.lemmatize(lemmatizer.lemmatize(x,pos='a'))).lower() for x in word_tokenize(y) if x not in stopwords.words("english")])) for y in texts]

# A few examples:
print(texts[0])
print(processed_texts[0])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
go jurong point , crazy.. avail bugi n great world la e buffet ... cine got amor wat ...


In [35]:
X_proc_tf = tfidfv.fit_transform(processed_texts)
X_proc_cv = sfecv.fit_transform(processed_texts)

Результаты на кросс-валидации с наилучшими параметрами.

In [36]:
print('CountVectorizer:', cross_val_score(lrc,X_proc_cv,target,cv=10, scoring='f1').mean())
print('TF-IDF:', cross_val_score(lrc,X_proc_tf,target,cv=10, scoring='f1').mean())

CountVectorizer: 0.934619191385
TF-IDF: 0.83999258071


In [38]:
gscv.fit(X_proc_cv, target)
print('CountVectorizer on tuned LogisticRegression:', cross_val_score(gscv.best_estimator_,X_proc_cv,target,cv=10, scoring='f1').mean())
print('Best params:', gscv.best_params_)

CountVectorizer on tuned LogisticRegression: 0.943519745604
Best params: {'C': 100.0, 'penalty': 'l2'}


При использовании CountVestorizer улучшение незначительно. При использовании TF-IDF наблюдается снижение качества классификации.
Используем удаление стоп-слов + приведение к нижнему регистру + стэмминг:

In [31]:
processed_texts_2 = [(' '.join([ps.stem(x).lower() for x in word_tokenize(y) if x not in stopwords.words("english")])) for y in texts]

In [39]:
lrc_best = LogisticRegression(C=100, penalty='l2')
X_proc_cv = sfecv.fit_transform(processed_texts_2)
print('CountVectorizer', cross_val_score(lrc_best,X_proc_cv,target,cv=10, scoring='f1').mean())

CountVectorizer 0.943519745604


Используем удаление стоп-слов + приведение к нижнему регистру + лемматизацию:

In [40]:
processed_texts_3 = [(' '.join([lemmatizer.lemmatize(x).lower() for x in word_tokenize(y) if x not in stopwords.words("english")])) for y in texts]

In [41]:
X_proc_cv = tfidfv.fit_transform(processed_texts_3)
print('CountVectorizer', cross_val_score(lrc_best,X_proc_cv,target,cv=10, scoring='f1').mean())

CountVectorizer 0.930783911385


## Выводы

- Для задачи бинарной классификации текстов SMS-сообщений CountVectorizer лучше подходит в качестве способа отбора признаков, нежели TfidfVectorizer.
- Линейная модель логистической регрессии лучше справилась с задачей классификации, чем наивный Байесовский классификатор (в обоих случаях использовались стандартные параметры).
- Предварительная обработка текста, такая как, например, приведение к нижнему регистру, стэмминг и лемматизация, может несколько повысить точность классификации.